In [1]:
import os
from estnltk import Text
from estnltk_neural.taggers import StanzaSyntaxTagger
import pandas as pd
import copy
import numpy as np
import sklearn
import csv
import sqlite3
from estnltk.converters import text_to_json
import json

Alustuseks fraasimustrid, mis peaksid olema korrektsed. Fraasimustrid on tekstifailis, igal real üks muster. Esituskuju on järgnev: id-ner/timex/other-ülemuse_id-deprel-POS eraldatuna tühikutega. Eri sõnade info on eraldatud komadega. Juhul, kui POS on mitmene, esitatakse variandid püstkriipsuga eraldatult. Indekseid mustrites ei esitata, sest fraasisiseste otsingut korral võivad tekkida ebakõlad. Liikmete järjestus vastab järjestusele tekstis.



In [2]:
# patterns are kept in a list of strings
patterns = None

with open('correct_tree_patterns.txt', 'r') as f:
    patterns = [line.rstrip() for line in f]

for pattern in patterns:
    print(pattern)

1 PER 0 root H,2 OTHER 1 flat H,3 OTHER 1 flat H
1 LOC 2 nmod G,2 OTHER 0 root S
1 LOC 2 nmod H,2 OTHER 0 root D
1 LOC 2 nmod H,2 OTHER 0 root H
1 LOC 2 nmod H,2 OTHER 0 root S
1 ORG 2 nmod S,2 OTHER 0 root S
1 ORG 2 nmod Y,2 OTHER 0 root S
1 LOC 2 amod G,2 OTHER 0 root S
1 LOC 2 amod H,2 OTHER 0 root S
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 ORG 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat S
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat Y
1 LOC 2 nmod H,2 OTHER 0 root S,3 OTHER 2 appos S,4 OTHER 3 flat H
1 ORG 2 nmod S,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 ORG 2 nmod S,2 OTHER 0 root S,3 OTHER 2 appos S,4 OTHER 3 flat H
1 LOC 2 nmod Y,2 OTHER 0 root S,3 OTHER 2 appos H,4 OTHER 3 flat H
1 LOC 2 obl H,2 OTHER 3 acl A,3 OTHER 0 root S
1 PER 2 nmod H,2 OTHER 0 root S,3 OTHER 2 nmod S
1 LOC 2 obl A,2 OTHER 3 acl A,3 OTHER 0 root S
1 LOC 2 obl Y,2 OTHER 3 acl A,3 OTHER 0 root S
1 OTHER 3 advmod D|I,2 PER 3 nmod H,

In [10]:
patterns_simplified = []

for pattern in patterns:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(1)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_simplified.append(','.join(pieces))
    
for pattern in patterns_simplified:
    print(pattern)

1 0 root H,2 1 flat H,3 1 flat H
1 2 nmod G,2 0 root S
1 2 nmod H,2 0 root D
1 2 nmod H,2 0 root H
1 2 nmod H,2 0 root S
1 2 nmod S,2 0 root S
1 2 nmod Y,2 0 root S
1 2 amod G,2 0 root S
1 2 amod H,2 0 root S
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat H
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat S
1 2 nmod H,2 0 root S,3 2 appos H,4 3 flat Y
1 2 nmod H,2 0 root S,3 2 appos S,4 3 flat H
1 2 nmod S,2 0 root S,3 2 appos H,4 3 flat H
1 2 nmod S,2 0 root S,3 2 appos S,4 3 flat H
1 2 nmod Y,2 0 root S,3 2 appos H,4 3 flat H
1 2 obl H,2 3 acl A,3 0 root S
1 2 nmod H,2 0 root S,3 2 nmod S
1 2 obl A,2 3 acl A,3 0 root S
1 2 obl Y,2 3 acl A,3 0 root S
1 3 advmod D|I,2 3 nmod H,3 0 root S
1 3 advmod D,2 3 nmod H,3 0 root S
1 3 advmod S,2 3 nmod H,3 0 root S
1 3 amod H,2 3 amod A,3 0 root S
1 3 amod H,2 3 amod O,3 0 root S
1 3 amod A,2 3 nmod G,3 0 root S
1 3 amod A,2 3 nmod H,3 0 root S
1 3 amod A,2 3 nmod Y,3 0 root S
1 3 amod O,2 3 nmod H,3 0 root S
1 3 det P|N,2 3 nmod H,3 0 root S
1 3 det A,2

In [69]:
patterns_with_ner_timex_other = []

for pattern in patterns:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(4)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_with_ner_timex_other.append(','.join(pieces))
    
for pattern in patterns_with_ner_timex_other:
    print(pattern)

1 PER 0 root,2 OTHER 1 flat,3 OTHER 1 flat
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root
1 ORG 2 nmod,2 OTHER 0 root
1 ORG 2 nmod,2 OTHER 0 root
1 LOC 2 amod,2 OTHER 0 root
1 LOC 2 amod,2 OTHER 0 root
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 ORG 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 nmod,2 OTHER 0 root,3 OTHER 2 appos,4 OTHER 3 flat
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 PER 2 nmod,2 OTHER 0 root,3 OTHER 2 nmod
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 LOC 2 obl,2 OTHER 3 acl,3 OTHER 0 root
1 OTHER 3 advmod,2 PER 3 nmod,3 OTHER 0 root
1 OTHER 3 advmod,2 LOC 3 nmod,3 OTHER 0 root
1 OTHER 3 advmod,2 LOC 3 nmod,3 OTHER 0 root
1 LOC 3 amod,2 OTHE

In [14]:
patterns_more_simplified = []

for pattern in patterns_simplified:
    pieces = [wrd.split() for wrd in pattern.split(',')]
    for piece in pieces:
        piece.pop(3)

    for i in range(len(pieces)):
        pieces[i] = ' '.join(pieces[i])
    patterns_more_simplified.append(','.join(pieces))
    
for pattern in patterns_more_simplified:
    print(pattern)

1 0 root,2 1 flat,3 1 flat
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 nmod,2 0 root
1 2 amod,2 0 root
1 2 amod,2 0 root
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 nmod,2 0 root,3 2 appos,4 3 flat
1 2 obl,2 3 acl,3 0 root
1 2 nmod,2 0 root,3 2 nmod
1 2 obl,2 3 acl,3 0 root
1 2 obl,2 3 acl,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 advmod,2 3 nmod,3 0 root
1 3 amod,2 3 amod,3 0 root
1 3 amod,2 3 amod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 amod,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 det,2 3 nmod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nmod,2 3 amod,3 0 root
1 3 nm

In [236]:
df = pd.read_pickle('phrase_examples_filtered_sub10000/non_atomic_phrases.pkl')

In [237]:
df

,phrase,phrase_length,document_creation_time,sentence_id,document_id,sentence_startend,subcorpus,phrase_type,phrase_start_end,has_ner_netity,...,pos_sequence,graph,graph_code,graph_code_pos,graph_code_pos_ner_timex,pos_sequence_verb_info,graph_verb_info,graph_code_verb_info,graph_code_pos_verb_info,graph_code_pos_ner_timex_verb_info
0,Text(text='neist teede äärtesse'),3,2024-03-10T00:43,5858921,312319,"(0, 226)",aja_EPL,obl_phrase,"(84, 104)",0,...,P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))",P-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, det),(3, 2, nmod))","(S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))","(0-0-0,S-P-S,(0, 3, root),(2, 1, det),(3, 2, nmod))"
1,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),9,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nsubj_cop_phrase,"(0, 76)",1,...,H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))",H-S-S-S-S-S-S-H-H,"(1, 2, 3, 4, 5, 6, 7, 0, 8, 9)","((0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))","(0-ner_ORG-ner_ORG-0-0-0-0-0-0,S-H-S-S-S-S-S-H-H,(0, 7, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod),(7, 6, nmod),(7, 8, appos),(8, 9, flat))"
2,Text(text='Tartu ülikooli ajaloo'),3,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 21)",1,...,H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))",H-S-S,"(1, 2, 3, 0)","((0, 3, root),(2, 1, nmod),(3, 2, nmod))","(S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))","(0-ner_ORG-ner_ORG,S-H-S,(0, 3, root),(2, 1, nmod),(3, 2, nmod))"
3,Text(text='Tartu ülikooli ajaloo osakonna'),4,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 30)",1,...,H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))",H-S-S-S,"(1, 2, 3, 4, 0)","((0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))","(0-ner_ORG-ner_ORG-0,S-H-S-S,(0, 4, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod))"
4,Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli'),6,2024-03-10T00:43,11085184,487500,"(227, 375)",aja_pm,nmod_phrase,"(0, 50)",1,...,H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))",H-S-S-S-S-S,"(1, 2, 3, 4, 5, 6, 0)","((0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))","(0-ner_ORG-ner_ORG-0-0-0,S-H-S-S-S-S,(0, 6, root),(2, 1, nmod),(3, 2, nmod),(4, 3, nmod),(5, 4, nmod),(6, 5, nmod))"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3051,Text(text='konkureeriva kaabeltelevisooni firma'),3,2024-03-10T01:35,1526710,61973,"(522, 718)",aja_

In [238]:
test = None
for idx, row in df.iterrows():
    test = row['phrase']
    break

In [239]:
test.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('neist', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pl', 'pl'), ('el', 'el')]), 'head': 2, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])])

In [4]:
def get_ner_timex(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    timex = text_obj.timexes.get(stanza_word)
    if ner:
        return nertag
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [60]:
def get_ner(text_obj, stanza_word):
    ner = None
    nertag = None
    if len(text_obj.ner) > 0:
        word = text_obj.words.get(stanza_word)
        for n in text_obj.ner:
            for part in n:
                if part==word:
                    ner=word
                    nertag=n.nertag
    if ner:
        return nertag
    return 'OTHER'

In [64]:
def get_timex(text_obj, stanza_word):
    timex = text_obj.timexes.get(stanza_word)
    if timex:
        return 'TIMEX'
    return 'OTHER'

In [5]:
def get_unique_POS(word):
    #pos_list = word.morph_analysis['partofspeech']
    infinite_verb_forms = ['da', 'des', 'ma', 'maks', 'mas', 'mast', 'mata', 'nud', 'tav', 'tud', 'v']
    # if POS is ambiguous, only unique tags are kept, e.g. ['V', 'A', 'A'] -> ['V', 'A']
    pos_list = []
    for i in range(len(word.morph_analysis['partofspeech'])):
        if word.morph_analysis['partofspeech'][i] == 'V':
            if word.morph_analysis['form'][i] in infinite_verb_forms:
                pos_list.append('V_inf')
            elif word.form[i] == 'neg':
                pos_list.append('V_neg')
            else:
                pos_list.append('V_fin')
        else:
            pos_list.append(word.morph_analysis['partofspeech'][i])
    
    if len(pos_list) > 1:
        char_unique = [char for indx, char in enumerate(pos_list) if char not in pos_list[:indx]]
        if len(char_unique) < 2:
            return char_unique[0]
        return '|'.join(char_unique)
    return pos_list[0]

In [30]:
test = Text('Nad käisid eile koolides').tag_layer('morph_analysis')
test.morph_analysis

Layer(name='morph_analysis', attributes=('normalized_text', 'lemma', 'root', 'root_tokens', 'ending', 'clitic', 'form', 'partofspeech'), spans=SL[Span('Nad', [{'normalized_text': 'Nad', 'lemma': 'tema', 'root': 'tema', 'root_tokens': ['tema'], 'ending': 'd', 'clitic': '', 'form': 'pl n', 'partofspeech': 'P'}]),
Span('käisid', [{'normalized_text': 'käisid', 'lemma': 'käima', 'root': 'käi', 'root_tokens': ['käi'], 'ending': 'sid', 'clitic': '', 'form': 'sid', 'partofspeech': 'V'}]),
Span('eile', [{'normalized_text': 'eile', 'lemma': 'eile', 'root': 'eile', 'root_tokens': ['eile'], 'ending': '0', 'clitic': '', 'form': '', 'partofspeech': 'D'}]),
Span('koolides', [{'normalized_text': 'koolides', 'lemma': 'kool', 'root': 'kool', 'root_tokens': ['kool'], 'ending': 'des', 'clitic': '', 'form': 'pl in', 'partofspeech': 'S'}])])

In [31]:
def get_word_form(word):
    pieces = word.morph_analysis['form'][0].split()
    if len(pieces) < 1:
        return '0'
    else:
        if len(pieces) > 1:
            return pieces[1]
        else:
            return pieces[0]

In [32]:
for word in test.words:
    print(get_word_form(word))

n
sid
0
in


Naiivne lähenemine: vaatab vaid variante, kus liikmed paiknevad järjest:

In [83]:
pattern_matches_found = []

for i in range(len(test.stanza_syntax)-1):
    pattern_found = []
    ner_timex = get_ner_timex(test, test.stanza_syntax[i])
    pos = get_unique_POS(test.stanza_syntax[i])
    pattern_found.append(f'{test.stanza_syntax[i].id} {ner_timex} {test.stanza_syntax[i].head} {test.stanza_syntax[i].deprel} {pos}')
    for j in range(i+1, len(test.stanza_syntax)):
        ner_timex = get_ner_timex(test, test.stanza_syntax[j])
        pos = get_unique_POS(test.stanza_syntax[j])
        pattern_found.append(f'{test.stanza_syntax[j].id} {ner_timex} {test.stanza_syntax[j].head} {test.stanza_syntax[j].deprel} {pos}')
        if ','.join(pattern_found) in patterns:
            pattern_matches_found += ','.join(pattern_found)
        if j == len(test.stanza_syntax):
            pattern_found = []
            break
        

In [84]:
print(','.join(pattern_found))

2 ORG 1 appos H,3 OTHER 2 flat S


In [249]:
test2 = None
for idx, row in df.iterrows():
    if idx == 0:
        test2 = row['phrase']
        break

In [250]:
test2.stanza_syntax

Layer(name='stanza_syntax', attributes=('id', 'lemma', 'upostag', 'xpostag', 'feats', 'head', 'deprel', 'deps', 'misc', 'parent_span', 'children'), spans=SL[Span('neist', [{'id': 1, 'lemma': 'see', 'upostag': 'P', 'xpostag': 'P', 'feats': OrderedDict([('pl', 'pl'), ('el', 'el')]), 'head': 2, 'deprel': 'det', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]),
Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]),
Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])])

In [312]:
# getting patterns without ner/timex/other
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    #ner_timex = get_ner_timex(test2, test2.stanza_syntax[i])
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                #ner_timex = get_ner_timex(test2, test2.stanza_syntax[j])
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append([",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break

print(pattern_matches_found)

['1 2 det P', '2 0 root S']
1 2 det P,2 0 root S
False
['1 2 det P', '2 3 nmod S', '3 0 root S']
1 2 det P,2 3 nmod S,3 0 root S
False
['1 2 nmod S', '2 0 root S']
1 2 nmod S,2 0 root S
True
[['1 2 nmod S,2 0 root S', 'teede äärtesse', [Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]]]


In [313]:
# getting patterns with ner/timex/other
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ner_timex = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[i]))
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[j]))
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), ner_timex[l], str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append(",".join(sub_pattern))

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break
                

['1 OTHER 2 det P', '2 OTHER 0 root S']
1 OTHER 2 det P,2 OTHER 0 root S
False
['1 OTHER 2 det P', '2 OTHER 3 nmod S', '3 OTHER 0 root S']
1 OTHER 2 det P,2 OTHER 3 nmod S,3 OTHER 0 root S
False
['1 OTHER 2 nmod S', '2 OTHER 0 root S']
1 OTHER 2 nmod S,2 OTHER 0 root S
False


### On more data

In [38]:
def find_patterns(phrase):
    pattern_matches_found = []
    
    for i in range(len(phrase.stanza_syntax) - 1):
        pattern_members = []
        ids = []

        pattern_members.append(phrase.stanza_syntax[i])
        ids.append([phrase.stanza_syntax[i].id, phrase.stanza_syntax[i].head])

        for j in range(i + 1, len(phrase.stanza_syntax)):
            pattern_found = []
            for k in range(len(pattern_members)):
                if phrase.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in phrase.stanza_syntax[j].children or phrase.stanza_syntax[j].parent != None and phrase.stanza_syntax[j].parent == pattern_members[k].parent:
                    pattern_members.append(phrase.stanza_syntax[j])
                    ids.append([phrase.stanza_syntax[j].id, phrase.stanza_syntax[j].head])
        
            ids_for_pattern = copy.deepcopy(ids)
            for l in range(len(ids_for_pattern)):
                temp = ids_for_pattern[l][0]
                ids_for_pattern[l][0] = l+1
                for m in range(len(ids)):
                    if ids[m][1] == temp:
                        ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
            word_ids = [word_id[0] for word_id in ids_for_pattern]
            for l in range(len(ids_for_pattern)):
                if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                    ids_for_pattern[l][1] = 0
                elif ids_for_pattern[l][1] not in word_ids:
                    ids_for_pattern[l][1] = 0
            
            for l in range(len(pattern_members)):
                deprel = pattern_members[l].deprel
                if ids_for_pattern[l][1] == 0 and deprel != 'root':
                    deprel = 'root'
                pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel])
                    
            sub_pattern = [" ".join(word_info) for word_info in pattern_found]
            if ",".join(sub_pattern) in patterns_more_simplified:
                pattern_matches_found.append([",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

            if j == len(phrase.stanza_syntax) - 1:
                pattern_members = []
                ids = []
                break             
                    
    return pattern_matches_found

In [299]:
def find_patterns_pos(phrase):
    pattern_matches_found = []
    
    for i in range(len(phrase.stanza_syntax) - 1):
        pattern_members = []
        ids = []

        pattern_members.append(phrase.stanza_syntax[i])
        #ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[i])
        ids.append([phrase.stanza_syntax[i].id, phrase.stanza_syntax[i].head])

        for j in range(i + 1, len(phrase.stanza_syntax)):
            pattern_found = []
            for k in range(len(pattern_members)):
                if phrase.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in phrase.stanza_syntax[j].children or phrase.stanza_syntax[j].parent != None and phrase.stanza_syntax[j].parent == pattern_members[k].parent:
                    pattern_members.append(phrase.stanza_syntax[j])
                    #ner_timex = get_ner_timex(phrase, phrase.stanza_syntax[j])
                    ids.append([phrase.stanza_syntax[j].id, phrase.stanza_syntax[j].head])
        
            ids_for_pattern = copy.deepcopy(ids)
            for l in range(len(ids_for_pattern)):
                temp = ids_for_pattern[l][0]
                ids_for_pattern[l][0] = l+1
                for m in range(len(ids)):
                    if ids[m][1] == temp:
                        ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
            word_ids = [word_id[0] for word_id in ids_for_pattern]
            for l in range(len(ids_for_pattern)):
                if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                    ids_for_pattern[l][1] = 0
                elif ids_for_pattern[l][1] not in word_ids:
                    ids_for_pattern[l][1] = 0
            
            for l in range(len(pattern_members)):
                deprel = pattern_members[l].deprel
                if ids_for_pattern[l][1] == 0 and deprel != 'root':
                    deprel = 'root'
                pattern_found.append([str(ids_for_pattern[l][0]), str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
                    
            sub_pattern = [" ".join(word_info) for word_info in pattern_found]
            if ",".join(sub_pattern) in patterns_simplified:
                pattern_matches_found.append([",".join(sub_pattern), " ".join([member.text for member in pattern_members]), [member for member in pattern_members]])

            if j == len(phrase.stanza_syntax) - 1:
                pattern_members = []
                ids = []
                break             
                    
    return pattern_matches_found
                

In [ ]:
def find_patterns_ner_timex(phrase):
pattern_matches_found = []

for i in range(len(test2.stanza_syntax) - 1):
    pattern_members = []
    ner_timex = []
    ids = []

    pattern_members.append(test2.stanza_syntax[i])
    ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[i]))
    ids.append([test2.stanza_syntax[i].id, test2.stanza_syntax[i].head])

    for j in range(i + 1, len(test2.stanza_syntax)):
        pattern_found = []
        current_members = []
        for k in range(len(pattern_members)):
            #print(member)
            #print(test2.stanza_syntax[j])
            if test2.stanza_syntax[j] in pattern_members[k].children or pattern_members[k] in test2.stanza_syntax[j].children or test2.stanza_syntax[j].parent != None and test2.stanza_syntax[j].parent == pattern_members[k].parent:
                current_members.append(pattern_members[k])
                current_members.append(test2.stanza_syntax[j])
                pattern_members.append(test2.stanza_syntax[j])
                ner_timex.append(get_ner_timex(test2, test2.stanza_syntax[j]))
                ids.append([test2.stanza_syntax[j].id, test2.stanza_syntax[j].head])
        
        ids_for_pattern = copy.deepcopy(ids)
        for l in range(len(ids_for_pattern)):
            temp = ids_for_pattern[l][0]
            ids_for_pattern[l][0] = l+1
            for m in range(len(ids)):
                if ids[m][1] == temp:
                    ids_for_pattern[m][1] = ids_for_pattern[l][0]
            
        word_ids = [word_id[0] for word_id in ids_for_pattern]
        for l in range(len(ids_for_pattern)):
            if ids_for_pattern[l][0] == ids_for_pattern[l][1]:
                ids_for_pattern[l][1] = 0
            elif ids_for_pattern[l][1] not in word_ids:
                ids_for_pattern[l][1] = 0
            
        for l in range(len(pattern_members)):
            deprel = pattern_members[l].deprel
            if ids_for_pattern[l][1] == 0 and deprel != 'root':
                deprel = 'root'
            pattern_found.append([str(ids_for_pattern[l][0]), ner_timex[l], str(ids_for_pattern[l][1]), deprel, get_unique_POS(pattern_members[l])])
        
        sub_pattern = [" ".join(word_info) for word_info in pattern_found]
        print(sub_pattern)
        print(",".join(sub_pattern))
        print(",".join(sub_pattern) in patterns_simplified)
        if ",".join(sub_pattern) in patterns_simplified:
            pattern_matches_found.append(",".join(sub_pattern))

        if j == len(test2.stanza_syntax) - 1:
            pattern_members = []
            ids = []
            break

In [51]:
# searching for matches from both atomic and non-atomic phrases
df_non_atomic = pd.read_pickle('phrase_examples_filtered_sub10000/non_atomic_phrases.pkl')
df_atomic = pd.read_pickle('phrase_examples_filtered_sub10000/atomic_phrases.pkl')
concatenated = pd.concat([df_non_atomic, df_atomic])

correct_matches = {'pattern_tree': [], 'phrase_text': [], 'phrase_spans': [], 'parent_phrase': []}

for idx, row in concatenated.iterrows():
    matches = find_patterns(row['phrase'])
    if len(matches) > 0:
        for match in matches:
            correct_matches['pattern_tree'].append(match[0])
            correct_matches['phrase_text'].append(match[1])
            correct_matches['phrase_spans'].append(match[2])
            correct_matches['parent_phrase'].append(row['phrase'])
            
matches_df = pd.DataFrame(correct_matches)

In [52]:
matches_df

,pattern_tree,phrase_text,phrase_spans,parent_phrase
0,"1 2 nmod,2 0 root",teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,"1 2 nmod,2 0 root",Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,"1 2 nmod,2 0 root",ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,"1 2 nmod,2 0 root",ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,"1 2 nmod,2 0 root",osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...
21487,"1 2 nmod,2 0 root",õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('subjekt', [{'id': 2, 'lemma': 'subjekt', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='õiguse subjekt')
21488,"1 2 nmod,2 0 root",Pekingi slummis,"[Span('Pekingi', [{'id': 1, 'lemma': 'Peking', 'upostag': '

In [53]:
# NER/TIMEX/OTHER
combinations = []
for idx, row in matches_df.iterrows():
    combination = []
    for stanza_word in row['phrase_spans']:
        combination.append(get_ner_timex(row['parent_phrase'], stanza_word))
    combinations.append("-".join(combination))

matches_df.insert(0, 'ner/timex/other', combinations)

In [54]:
matches_df

,ner/timex/other,pattern_tree,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('subjekt', [{'id': 2, 'lemma': 'subjekt', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='õiguse subjekt')
21488,OTHER-OTHE

In [55]:
# POS-sequences
sequences = []
for idx, row in matches_df.iterrows():
    sequence = []
    for stanza_word in row['phrase_spans']:
        sequence.append(get_unique_POS(stanza_word))
    sequences.append("-".join(sequence))

matches_df.insert(2, 'pattern_pos', sequences)

In [56]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('subjekt', [{'id': 2, 'lemma': 'subjekt', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text

In [58]:
# word forms
forms = []
for idx, row in matches_df.iterrows():
    form = []
    for stanza_word in row['phrase_spans']:
        form.append(get_word_form(stanza_word))
    forms.append("-".join(form))

matches_df.insert(3, 'pattern_form', forms)

In [59]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,pattern_form,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-ill,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,g-g,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,g-g,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-n,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('subjekt', [{'id': 2, 'lemma': 'subjekt', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('n', 'n')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': 

In [62]:
# NER
combinations_ner = []
for idx, row in matches_df.iterrows():
    combination_ner = []
    for stanza_word in row['phrase_spans']:
        combination_ner.append(get_ner(row['parent_phrase'], stanza_word))
    combinations_ner.append("-".join(combination_ner))

matches_df.insert(4, 'pattern_ner', combinations_ner)

In [65]:
# TIMEX
combinations_timex = []
for idx, row in matches_df.iterrows():
    combination_timex = []
    for stanza_word in row['phrase_spans']:
        combination_timex.append(get_timex(row['parent_phrase'], stanza_word))
    combinations_timex.append("-".join(combination_timex))

matches_df.insert(5, 'pattern_timex', combinations_timex)

In [66]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,pattern_form,pattern_ner,pattern_timex,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-ill,OTHER-OTHER,OTHER-OTHER,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,g-g,ORG-ORG,OTHER-OTHER,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,g-g,ORG-OTHER,OTHER-OTHER,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,OTHER-OTHER,OTHER-OTHER,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,OTHER-OTHER,OTHER-OTHER,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-n,OTHER-OTHER,OTHER-OTHER,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('subjekt', [{'id': 2, 

In [49]:
#matches_df = matches_df.drop(columns=['pos_sequence'])

In [71]:
# adding information about pattern status (with ner/timex) -> 0 (pattern does not exist in pattern list), 
# 1 (pattern exists in pattern list)
pattern_statuses = []
for idx, row in matches_df.iterrows():
    ner_timex = row['ner/timex/other'].split('-')
    pattern_splitted = [part.split() for part in row['pattern_tree'].split(',')]
    for i in range(len(pattern_splitted)):
        pattern_splitted[i].insert(1, ner_timex[i])
    for i in range(len(pattern_splitted)):
        pattern_splitted[i] = " ".join(pattern_splitted[i])
    new_pattern = ",".join(pattern_splitted)
    
    if new_pattern in patterns_with_ner_timex_other:
        pattern_statuses.append(1)
    else:
        pattern_statuses.append(0)

matches_df.insert(6, 'pattern_status_ner_timex_other', pattern_statuses)


In [72]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,pattern_form,pattern_ner,pattern_timex,pattern_status_ner_timex_other,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-ill,OTHER-OTHER,OTHER-OTHER,0,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,g-g,ORG-ORG,OTHER-OTHER,0,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,g-g,ORG-OTHER,OTHER-OTHER,1,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,OTHER-OTHER,OTHER-OTHER,0,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,OTHER-OTHER,OTHER-OTHER,0,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-n,OTHER-OTHER,OTHER-OTHER,0,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>,

In [73]:
# adding column with start and end positions of phrases
phrase_startends = []
for idx, row in matches_df.iterrows():
    start = row['phrase_spans'][0].start
    end = row['phrase_spans'][-1].end
    phrase_startends.append([start, end])
    
matches_df.insert(4, 'startend', phrase_startends)

In [78]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,pattern_form,startend,pattern_ner,pattern_timex,pattern_status_ner_timex_other,phrase_text,phrase_spans,parent_phrase
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-ill,"[6, 20]",OTHER-OTHER,OTHER-OTHER,0,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse')
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,g-g,"[0, 14]",ORG-ORG,OTHER-OTHER,0,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[6, 21]",ORG-OTHER,OTHER-OTHER,1,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[15, 30]",OTHER-OTHER,OTHER-OTHER,0,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[22, 40]",OTHER-OTHER,OTHER-OTHER,0,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul')
...,...,...,...,...,...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-n,"[0, 14]",OTHER-OTHER,OTHER-OTHER,0,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps

In [79]:
unique_parent_phrases = []

for idx, row in matches_df.iterrows():
    if row['parent_phrase'] not in unique_parent_phrases:
        unique_parent_phrases.append(row['parent_phrase'])

In [81]:
len(unique_parent_phrases)

11333

In [82]:
# parent phrase IDs
parent_phrase_ids = []
for idx, row in matches_df.iterrows():
    for i in range(len(unique_parent_phrases)):
        if unique_parent_phrases[i] == row['parent_phrase']:
            parent_phrase_ids.append(i)

matches_df.insert(11, 'parent_phrase_id', parent_phrase_ids)

In [83]:
matches_df

,ner/timex/other,pattern_tree,pattern_pos,pattern_form,startend,pattern_ner,pattern_timex,pattern_status_ner_timex_other,phrase_text,phrase_spans,parent_phrase,parent_phrase_id
0,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-ill,"[6, 20]",OTHER-OTHER,OTHER-OTHER,0,teede äärtesse,"[Span('teede', [{'id': 2, 'lemma': 'tee', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('äärtesse', [{'id': 3, 'lemma': 'äär', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('pl', 'pl'), ('ill', 'ill')]), 'head': 0, 'deprel': 'root', 'deps': '_', 'misc': '_', 'parent_span': None, 'children': <class 'tuple'>}])]",Text(text='neist teede äärtesse'),0
1,ORG-ORG,"1 2 nmod,2 0 root",H-S,g-g,"[0, 14]",ORG-ORG,OTHER-OTHER,0,Tartu ülikooli,"[Span('Tartu', [{'id': 1, 'lemma': 'Tartu', 'upostag': 'H', 'xpostag': 'H', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 2, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': ()}]), Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),1
2,ORG-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[6, 21]",ORG-OTHER,OTHER-OTHER,1,ülikooli ajaloo,"[Span('ülikooli', [{'id': 2, 'lemma': 'ülikool', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 3, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),1
3,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[15, 30]",OTHER-OTHER,OTHER-OTHER,0,ajaloo osakonna,"[Span('ajaloo', [{'id': 3, 'lemma': 'ajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 4, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),1
4,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-g,"[22, 40]",OTHER-OTHER,OTHER-OTHER,0,osakonna üldajaloo,"[Span('osakonna', [{'id': 4, 'lemma': 'osakond', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 5, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}]), Span('üldajaloo', [{'id': 5, 'lemma': 'üldajalugu', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'head': 6, 'deprel': 'nmod', 'deps': '_', 'misc': '_', 'parent_span': <class 'estnltk_core.layer.span.Span'>, 'children': <class 'tuple'>}])]",Text(text='Tartu ülikooli ajaloo osakonna üldajaloo õppetooli vanemlaborant Sirje Tamul'),1
...,...,...,...,...,...,...,...,...,...,...,...,...
21487,OTHER-OTHER,"1 2 nmod,2 0 root",S-S,g-n,"[0, 14]",OTHER-OTHER,OTHER-OTHER,0,õiguse subjekt,"[Span('õiguse', [{'id': 1, 'lemma': 'õigus', 'upostag': 'S', 'xpostag': 'S', 'feats': OrderedDict([('sg', 'sg'), ('g', 'g')]), 'he

In [84]:
con = sqlite3.connect("correct_noun_phrases.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE correct_phrase_patterns(ID INTEGER PRIMARY KEY, ner_timex_other TEXT, pattern_tree TEXT, pattern_pos TEXT, pattern_form TEXT, pattern_ner TEXT, pattern_timex TEXT, pattern_status_ner_timex_other INTEGER, phrase_text TEXT, phrase_word_list TEXT, startend TEXT, parent_phrase_id INTEGER)")

In [85]:
con.close()

In [88]:
con = sqlite3.connect("correct_noun_phrases.db")
cur = con.cursor()

for idx, row in matches_df.iterrows():
    phrase_word_list = row['phrase_text'].split()
    phrase_word_list_json = json.dumps(phrase_word_list)
    #parent_phrase_json = text_to_json(row['parent_phrase'])
    startend_str = str(row['startend'])
    cur.execute("""INSERT INTO correct_phrase_patterns
                            (ner_timex_other, pattern_tree, pattern_pos, pattern_form, pattern_ner, pattern_timex, pattern_status_ner_timex_other, phrase_text, phrase_word_list, startend, parent_phrase_id)
                            VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?);""", (row['ner/timex/other'], row['pattern_tree'], row['pattern_pos'], row['pattern_form'], row['pattern_ner'], row['pattern_timex'], row['pattern_status_ner_timex_other'], row['phrase_text'], phrase_word_list_json, startend_str, row['parent_phrase_id']))
    
    con.commit()

con.close()

In [91]:
con = sqlite3.connect("correct_parent_phrases.db")
cur = con.cursor()
cur.execute('pragma encoding=UTF8')
cur.execute("CREATE TABLE correct_parent_phrases(ID INTEGER PRIMARY KEY, phrase_id INTEGER, phrase TEXT)")

In [92]:
con.close()

In [94]:
con = sqlite3.connect("correct_parent_phrases.db")
cur = con.cursor()

for i in range(len(unique_parent_phrases)):
    parent_phrase_json = text_to_json(unique_parent_phrases[i])
    cur.execute("""INSERT INTO correct_parent_phrases
                            (phrase_id, phrase)
                            VALUES (?, ?);""", (i, parent_phrase_json))
    
    con.commit()

con.close()